In [1]:
%pip install pydub


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence

# Parameters
min_silence_len = 500  # Minimum length of silence to consider for splitting (ms)
silence_thresh_offset = 14  # Silence threshold offset (in dBFS)
padding_duration = 1000  # Duration of padding to add before and after the word (ms)
input_folder = "unprocessed_recorded_audio"  # Folder with the .wav files
output_folder = "processed_recorded_audio"  # Folder where output will be saved

def trim_leading_trailing_silence(segment, silence_thresh, max_silence_len):
    # Detect leading and trailing silence
    silent_chunks = detect_silence(segment, min_silence_len=100, silence_thresh=silence_thresh)
    
    if silent_chunks:
        # Assume the silence detected is before and after the word
        leading_silence_end = silent_chunks[0][1] if silent_chunks[0][0] == 0 else 0
        trailing_silence_start = silent_chunks[-1][0] if silent_chunks[-1][1] == len(segment) else len(segment)
        
        # Trim only the leading and trailing silence (up to max_silence_len)
        start = leading_silence_end
        end = trailing_silence_start

        if end - start < len(segment):
            # Ensure trimming is within bounds and retains the word
            start = max(0, start - max_silence_len)
            end = min(len(segment), end + max_silence_len)
            return segment[start:end]
    return segment

# Iterate over all .wav files in the input folder
for wav_file in os.listdir(input_folder):
    if wav_file.endswith(".wav"):
        wav_path = os.path.join(input_folder, wav_file)
        
        # Load the audio file
        audio = AudioSegment.from_file(wav_path)
        
        # Silence threshold based on the audio file
        silence_thresh = audio.dBFS - silence_thresh_offset
        
        # Split audio based on silence
        segments = split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
        
        # Create a subfolder for each audio file in the output folder
        word = os.path.splitext(wav_file)[0]  # Use the filename (without extension) as the folder name
        subfolder_path = os.path.join(output_folder, word)
        os.makedirs(subfolder_path, exist_ok=True)  # Create the subfolder if it doesn't exist
        
        # Process each segment and save them in the respective subfolder
        for i, segment in enumerate(segments):
            # Remove leading and trailing silence
            trimmed_segment = trim_leading_trailing_silence(segment, silence_thresh, padding_duration)

            # Add padding to the beginning and end of each segment
            start_padding = AudioSegment.silent(duration=padding_duration)
            end_padding = AudioSegment.silent(duration=padding_duration)
            final_segment = start_padding + trimmed_segment + end_padding

            # Generate the filename
            filename = f"{word}_{i+1:02d}.wav"
            file_path = os.path.join(subfolder_path, filename)

            # Export the final segment
            final_segment.export(file_path, format="wav")

        print(f"Processing of {wav_file} complete! Saved in {subfolder_path}")
        
print("All files processed!")


c:\Users\HP\anaconda3\envs\py310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Processing of Backward.wav complete! Saved in processed_recorded_audio\Backward
Processing of Bed.wav complete! Saved in processed_recorded_audio\Bed
Processing of Bird.wav complete! Saved in processed_recorded_audio\Bird
Processing of Dog.wav complete! Saved in processed_recorded_audio\Dog
Processing of Down.wav complete! Saved in processed_recorded_audio\Down
Processing of Eight.wav complete! Saved in processed_recorded_audio\Eight
Processing of Five.wav complete! Saved in processed_recorded_audio\Five
Processing of Follow.wav complete! Saved in processed_recorded_audio\Follow
Processing of Forward.wav complete! Saved in processed_recorded_audio\Forward
Processing of Four.wav complete! Saved in processed_recorded_audio\Four
Processing of Go.wav complete! Saved in processed_recorded_audio\Go
Processing of Happy.wav complete! Saved in processed_recorded_audio\Happy
Processing of House.wav complete! Saved in processed_recorded_audio\House
Processing of Learn.wav complete! Saved in proce